# 

In [1]:
import numpy as np
import copy
import torch
from transformers import logging
logging.set_verbosity_error()

from PIL import Image
import matplotlib.pyplot as plt

from fromage import models
from fromage import utils

In [2]:
# Load model used in the paper.
model_dir = '/home/ubuntu/Project/Project_final/11-777-Project/fromage/runs/frozen_15'
model = models.load_fromage(model_dir,type1='ss')

Using HuggingFace AutoFeatureExtractor for openai/clip-vit-large-patch14.
Using facebook/opt-1.3b for the language model.
Using openai/clip-vit-large-patch14 for the visual model with 4 visual tokens.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


Freezing the LM.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
import json
import random
import os
dataset1 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final_1.json", "r"))
dataset2 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final_2.json", "r"))
dataset3 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final.json", "r"))

In [4]:
len(dataset1), len(dataset2), len(dataset3)

(38, 38, 38)

In [6]:
def generate_answer(q1,q2,c1,c2,Q):
    prompt1= c1+c2+ 'Q: '+  q1+ '\nA:'
    A1=model.generate_for_images_and_texts([prompt1], num_words=40)
    # prompt2= c1+c2+ 'Q: '+  q2+ '\nA:'
    # A2=model.generate_for_images_and_texts([prompt2], num_words=40)
    # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
    # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
    return A1[0]

In [7]:
import pickle
import os
def generate_answer_img(q1,q2,c1,c2,Q,qid):
    if c2==None:
        gold_feature = []
        image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/'+str(qid)+'_0.pkl'
        image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/'+str(qid)+'_0.pkl'
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        prompt1= 'Q: '+  q1+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        # prompt2= 'Q: '+  q2+ '\nA:'
        # A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
        A_final=A1
    else:
        gold_feature = []
        image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/'+str(qid)+'_0.pkl'
        image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/'+str(qid)+'_1.pkl'
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        # prompt1= 'Q: '+  q1+ '\nA:'
        # A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        # prompt2= 'Q: '+  q2+ '\nA:'
        # A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
        prompt1= 'Q: '+  q1+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
    return A1[0]

In [8]:
Answers_real_im1 = []
Answers_pred_im1 = []
for i in dataset1:
    datum = dataset3[i]
    if len(datum['txt_posFacts']) == 0:
        q1= dataset3[i]['Q']
        q2= 'ss'
        if(len(dataset1[i]['img_posFacts'])==1):
            c1= dataset1[i]['img_posFacts'][0]['image_id']
            c2= None
        else:
            continue
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real_im1.append(A)
        Answers_pred_im1.append(generate_answer_img(q1,q2,c1,c2,Q,str(i)))

In [9]:
len(Answers_real_im1), len(Answers_pred_im1)

(11, 11)

In [10]:
Answers_pred_im1

[' "No.',
 ' "There are two total light posts at both ends of Abbey Road crosswalk in London.',
 ' "In a circle.',
 ' "In the left hand of the figure holding something.',
 ' "Red.',
 ' "The Lunar Module is on top of the Lunar Module.',
 ' "The USA"The USA was represented by the mascot, "The USA.',
 ' "There are two columns in front of the Cathedral of Blessed Sacrament in New Zealand.',
 ' "On the left side of the airplane\'s door.',
 ' "A circle.',
 ' "Yes.']

In [11]:
Answers_real_im = []
Answers_pred_im = []
for i in dataset1:
    datum = dataset1[i]
    if len(datum['txt_posFacts']) == 0:
        q1= dataset3[i]['Q']
        q2= ''
        if(len(dataset1[i]['img_posFacts'])<2):
            continue
        c1= dataset3[i]['img_posFacts'][0]['image_id']
        c2= dataset3[i]['img_posFacts'][1]['image_id']
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real_im.append(A)
        Answers_pred_im.append(generate_answer_img(q1,q2,c1,c2,Q,str(i)))

In [12]:
len(Answers_real_im), len(Answers_pred_im)

(14, 14)

In [13]:
Answers_pred_im

[' "Yes.',
 ' "Yes, there are steps at the entrance to the Hood Museum of Art and Budynek Biblioteki.',
 ' "No.',
 ' "The stalks of the Speise Morchel and the Amanita phalloides share a yellow color.',
 ' "No.',
 ' "Yes.',
 ' "No.',
 ' "The horse-tamer of the Monte Cavallo and a church and landscape with classical ruins and figures.',
 ' "Yes, both the Erica herbacea and the Lamium maculatum have spiky leaves.',
 ' "A horse.',
 ' "The Architecture Building in Toronto and the Danish Architecture Center share the same common features.',
 ' "No.',
 ' "No, the colors of the word lyric are the same in the Lyric Theater, Blacksburg and Lyric Theater, Georgia signs.',
 ' "The headlights on both the 1952 Kaiser Manhattan and the Lotus XI LeMans are round.']

In [14]:
Answers_real = []
Answers_pred = []
for i in dataset1:
    datum = dataset1[i]
    if len(datum['txt_posFacts']) != 0:
        q1= dataset3[i]['Q']
        q2= ''
        c1= dataset1[i]['txt_posFacts'][0]['fact']
        c2= dataset2[i]['txt_posFacts'][1]['fact']
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real.append(A)
        Answers_pred.append(generate_answer(q1,q2,c1,c2,Q))


In [15]:
len(Answers_real), len(Answers_pred)

(13, 13)

In [16]:
Answers_real=Answers_real+Answers_real_im+Answers_real_im1

Answers_pred=Answers_pred+Answers_pred_im+Answers_pred_im1

In [17]:
# save the pickle file
import pickle
with open('Answers_pred_nosubq.pkl', 'wb') as f:
    pickle.dump(Answers_pred, f)

In [18]:
Answers_real

['Australia.',
 'Think and Walking Away were released two years apart.',
 'West Virginia',
 'They are the same gender.',
 'teeth',
 'Genetics',
 'Pulmonary fibrosis affects the lungs, while Myelofibrosis affects the bone marrow.',
 'They are both an example of subfamily, under the larger name of Felidae.',
 'The West Rail line and the Mass Transit Railway both use light rails.',
 'The Palatine Hill',
 'No.',
 'No',
 'Both nerves are part of limbs.',
 '"Hang gliders and parachuters are not both attached to long chords."',
 '"No, there are not steps at the entrance of both the Hood Museum of Art and Budynek Biblioteki."',
 '"Yes, the hair part is lined up with the person\'s nose in both Her Know and Joseph Two Bulls."',
 '"They share the color white."',
 '"No, the seats at Baku Olympic Stadium are not the same color as the seats in the stadium at Olympic Park in Munich."',
 '"Yes, the Academy of Art University and The Hong Kong Academy for Performing Arts are both located on a hill."',
 

In [20]:
Answers_pred

[' Australia',
 ' "Think" was released in 1989 and "Walking Away" in 1990.',
 ' The Silver Bridge is located in Huntington, West Virginia, and the East Huntington Bridge is located in Gallipolis, Ohio.',
 ' The goddess Nut is a goddess of the sky, stars, cosmos, mothers, astronomy, and the universe in the ancient Egyptian religion.',
 ' The mandibles of both groups are modified to be used for chewing pollen grains.',
 ' Chromoprotein and Chromatic remodelling belong to the field of chromatin remodelling.',
 ' Yes.',
 ' The Pantherinae and Felinae are both subfamilies of felids, the family Felidae.',
 ' The West Rail line in Denver and the Mass Transit Railway in Hong Kong have the same type of rails.',
 ' From the Palatine Hill, you can see both the Basilica Julia and the Temple of Romulus.',
 ' Cangrande was a great warrior, and a great autocrat.',
 ' No.',
 ' The saphenous nerve is a branch of the subarticular branch of the saphenous nerve.',
 ' "Yes.',
 ' "Yes, there are steps at th

In [6]:
import pickle
import os
gold_feature = []
imgid_map=pickle.load(open('/home/ubuntu/Project/11-777-Project-aa/Data/image_id_map_0328.pkl', "rb"))
# image_id = imgid_map[image_id]
# image_feature_path =os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id%10000000)//1000, image_id))
image_feature_path='/home/ubuntu/Project/11-777-Project-aa/Data/dev/41/40041882.pkl'
gold_feature.append(image_feature_path)
gold_feature.append(image_feature_path)


In [23]:

for inp_text in ['The human history of Western Australia commenced between 40,000 and 60,000 years ago with the arrival of Indigenous Australians on the northwest coast. The first inhabitants expanded across the east and south of the continent. The North Shore is a region of Sydney, New South Wales, Australia, a subset of the Northern Sydney region. The region is customary, not legal or administrative, and generally refers to the suburbs located on the northern side of Sydney Harbour up to Waitara, which are between Middle Harbour and the Lane Cove River. Q: what is the name of the continent contains sydney harbour?\nA:']:
    prompt = [inp_text]
    model_outputs = model.generate_for_images_and_texts(prompt, num_words=40)

In [24]:
model_outputs 

[' Australia']

In [16]:
!gdown 1EqRblzhvchiN3_BGxLYGwtsoV-rHSK63

Downloading...
From: https://drive.google.com/uc?id=1EqRblzhvchiN3_BGxLYGwtsoV-rHSK63
To: /home/ubuntu/Project/Project_final/11-777-Project/fromage/question_answer_bao_chatgpt[1].json
100%|██████████████████████████████████████| 9.38k/9.38k [00:00<00:00, 28.3MB/s]


In [61]:
count=0
for i in dataset1:
    count+=1
    if count==20:
        q1=dataset1[i]['Q']
        q2=dataset2[i]['Q']
        Q=dataset3[i]['Q']
        c1=dataset1[i]['img_posFacts'][0]['url']
        c2=dataset1[i]['img_posFacts'][1]['url']
        prompt1= 'Q: '+  q1+ '\nA:'
        prompt2= 'Q: '+  q2+ '\nA:'
        print(prompt1)
        print(prompt2)
        print('Answer in separate lines')
        print(c1)
        print(c2)

Q: "Is the butt of the gun depicted on both the Civil War Memorial in Bethel, Maine ?"
A:
Q: "is the butt of the gun depicted on the one on the one on the Civil War Monument of Nuckolls County Courthouse touching the same surface that the soldiers' feet are touching?"
A:
Answer in separate lines
https://commons.wikimedia.org/wiki/File:Nuckolls_County_Courthouse_Civil_War_monument.JPG
https://commons.wikimedia.org/wiki/File:Civil_War_Memorial_in_Bethel%2C_Maine.jpg


def generate_questions(q1,q2,c1,c2,Q):
    prompt1= c1+c2+ 'Q: '+  q1+ '\nA:'
    prompt2= c1+c2+ 'Q: '+  q2+ '\nA:'
    prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
    return A_final[0]

In [58]:
a11_1='Yes, the Academy of Art University is located on a hill.'
a11_2='Yes, the Hong Kong Academy for Performing Arts is located on a hill.'

In [59]:
prompt3= 'Q: '+  q1+ ' \nA: ' +a11_1 + '\nQ: '+  q2+ ' \nA: ' +a11_2 + '\nQ: '+  Q+ ' \nA:'

In [60]:
prompt3

'Q: "Is the Academy of Art University located on a hill?" \nA: Yes, the Academy of Art University is located on a hill.\nQ: "Is The Hong Kong Academy for Performing Arts both located on a hill?" \nA: Yes, the Hong Kong Academy for Performing Arts is located on a hill.\nQ: "Is the Academy of Art University and The Hong Kong Academy for Performing Arts both located on a hill?" \nA:'

In [13]:
a_p=['  Australia',
 '  "Think" was released in 1989 and "Walking Away" was released in 1990.',
 '  West Virginia.',
 '  The goddess Nut is a goddess of the sky, stars, cosmos, mothers, astronomy, and the universe in the ancient Egyptian religion.',
 '  The mandible is the mouth part of the insect.',
 '  Chromatin remodelling is a field of science that deals with the dynamic modification of chromatin architecture to allow access of condensed genomic DNA to the regulatory transcription machinery proteins, and thereby control gene expression.',
 '  Yes, pulmonary fibrosis and Myelofibrosis affect the lungs, heart, liver, kidneys, and pancreas.',
 '  The Pantherinae and the Felinae are a subfamily of the Pantherinae, a family of large carnivores.',
 '  the West Rail line in Denver and the Mass Transit Railway in Hong Kong have a steel track.',
 '  from the Palatine Hill, the hill that is to the left of the Temple of Romulus.',
 '  Yes.',
 '  No.',
 '  The superior lateral brachial cutaneous nerve belongs to the subcutaneous branch of the saphenous nerve.']
answers_pred_univlr=a_p+AA_P


In [9]:
a_r_t=['Australia.',
 'Think and Walking Away were released two years apart.',
 'West Virginia',
 'They are the same gender.',
 'teeth',
 'Genetics',
 'Pulmonary fibrosis affects the lungs, while Myelofibrosis affects the bone marrow.',
 'They are both an example of subfamily, under the larger name of Felidae.',
 'The West Rail line and the Mass Transit Railway both use light rails.',
 'The Palatine Hill',
 'No.',
 'No',
 'Both nerves are part of limbs.']
Answers_real_univlr=AA_R+a_r_t

In [ ]:
A8/25

In [15]:
Answers_real_univlr

['"The female traditional clothing of the Sardinian people includes a skirt."',
 '"Two light posts are at both ends of Abbey Road crosswalk in London."',
 '"The petals of the flower of the Camellia Japonica are arranged in a star shape."',
 '"On the Berlin Tiergarten Denkmal Albrecht Graf von Roon statue, the figure is holding something in its left hand."',
 '"The predominant color of the Chacoan naked-tailed armadillo is brown."',
 '"Atop the Lunar Module in the Kennedy Space Center in Cape Canaveral, Florida, is a satellite dish."',
 '"A polar bear mascot was present at the 2014 Winter Olympics hockey game between the USA and Russia."',
 '"There are four columns in front of the Cathedral of Blessed Sacrament in New Zealand."',
 '"When the Prime Minister, Shri Narendra Modi, arrived at Mangalore airport in Karnataka on October 29, 2017 the Indian flag was to the right of the door."',
 '"The base of the statue in the Darlington Memorial Fountain is an octagon."',
 '"No, it does not hav

In [16]:
answers_pred_univlr 
# save as a pkl file
import pickle
with open('Answers_real_univlr.pkl', 'wb') as f:
    pickle.dump(Answers_real_univlr, f)